# Step 0: Importing required libraries

In [ ]:
# if you need to download uncomment this

#pip install beautifulsoup4

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict

In [ ]:
# if you need to download uncomment this

#nltk.download("punkt")
#nltk.download("cumdict")
#nltk.download('averaged_perceptron_tagger')

# Step 1: Data Extraction

In [ ]:
def data_extraction(link, id):
    #getting requests to access
    response = requests.get(link)

    #checking sucess of request
    if response.status_code == 200:
        # Parsing through the html doc
        soup = BeautifulSoup(response.text, 'html.parser')

        # extracting only article name and content
        tag_elements = soup.find_all('div', {'class': 'td-post-content tagdiv-type'})

        # condition for checking if tag exists in the html doc
        if tag_elements:
            # in case tag found extracting text content of article
            text_content = "\n".join(tag_element.get_text() for tag_element in tag_elements)

            # Saving the article content to a text file with the URL_ID as the filename
            with open(f"{id}.txt", "w", encoding="utf-8") as file:
                file.write(text_content)
            print(f"Successfully saved content for ID {id}")
        else:
          print(f"Failed to fetch content for ID {id}")

# loading excel file using pandas
excel_file = "Input.xlsx"
df = pd.read_excel(excel_file)

# loop to iterete over all URL_IDs
for index, row in df.iterrows():
    id = row['URL_ID']
    link = row['URL']
    data_extraction(link, id)


### Due to the use of different HTML tag in document some links wre not successfully parsed, after checking for required tag, the extraction code is re-executed for missing files:

In [ ]:
def data_extraction(link, id):
    #getting requests to access
    response = requests.get(link)

    #checking sucess of request
    if response.status_code == 200:
        # Parsing through the html doc
        soup = BeautifulSoup(response.text, 'html.parser')

        # extracting only article name and content
        tag_elements = soup.find_all('div', {'class': 'tdb-block-inner td-fix-index'})

        # condition for checking if tag exists in the html doc
        if tag_elements:
            # in case tag found extracting text content of article
            text_content = "\n".join(tag_element.get_text() for tag_element in tag_elements)

            # Saving the article content to a text file with the URL_ID as the filename
            with open(f"{id}.txt", "w", encoding="utf-8") as file:
                file.write(text_content)
            print(f"Successfully saved content for URL_ID {id}")
        else:
          print(f"Failed to fetch content for URL_ID {id}")

# Load the Excel file
#excel_file = "Input.xlsx"
#df = pd.read_excel(excel_file)

# List of missing URL IDs to process
specific_ids = ['blackassign0014','blackassign0020','blackassign0029','blackassign0043','blackassign0083','blackassign0084','blackassign0092','blackassign0099','blackassign0100']  # Replace with the IDs you want to process

# Iterate through each row in the Excel file
for index, row in df.iterrows():
    # Extract ID, link, and tag name from the current row
    id = row['URL_ID']
    link = row['URL']
    #tag_name = row['Tag']  # Assuming you have a column named 'Tag' in your Excel file

    # Check if the current ID is in the list of specific IDs
    if id in specific_ids:
        # Scrape website for specific tag and save content
        data_extraction(link, id)

# Step 2: Removing stopwords from given list of stopwords:

In [ ]:
try:
   # Loading all stopwords files
    def load_stopwords_from_files(stopwords_files):
        stopwords = set()
        for filepath in stopwords_files:
            with open(filepath, 'r', encoding='latin-1') as file:
                stopwords.update(file.read().splitlines())
        return stopwords

    # Function to remove stopwords from text
    def remove_stopwords(text, stopwords):
        return ' '.join([word for word in text.split() if word.lower() not in stopwords])

    # List of stopwords files
    stopwords_files = ['StopWords/StopWords_Auditor.txt', 'StopWords/StopWords_Currencies.txt', 'StopWords/StopWords_DatesandNumbers.txt','StopWords/StopWords_Generic.txt','StopWords/StopWords_GenericLong.txt','StopWords/StopWords_Geographic.txt','StopWords/StopWords_Names.txt']  # Add more files as needed

    # Load stopwords from files
    stopwords = load_stopwords_from_files(stopwords_files)

    # Iterate through each text file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            filepath = os.path.join(directory, filename)

            with open(filepath, 'r', encoding='latin-1') as file:
                text = file.read()

            # Removing stopwords from text
            text_without_stopwords = remove_stopwords(text, stopwords)

            # Writing text without stopwords
            with open(filepath, 'w', encoding='utf-8') as file:
                file.write(text_without_stopwords)
except Exception as e:
    error_message = str(e)
    print("An error occurred:", error_message)
else:
    print("Suceeded")

Suceeded


# Step 3: Data Analysis

## 1. Calulating:
### a)Positive Score
### b)Negative Score
### c)Polarity Score
### d)Subjectivity Score

In [ ]:
try:
    # Function to calculate positivity and negativity scores
    def calculate_scores(text, positive_words, negative_words):
        positivity_score = sum(1 for word in text.split() if word.lower() in positive_words)
        negativity_score = sum(1 for word in text.split() if word.lower() in negative_words)
        return positivity_score, negativity_score


    # Function to calculate Polarity Score
    def calculate_polarity_score(positive_score, negative_score):
        polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
        return polarity_score


    # Function to calculate Subjectivity Score
    def calculate_subjectivity_score(positive_score, negative_score, total_words):
        subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)
        return subjectivity_score


    # Read positive and negative word lists from Master Dictionary folder
    positive_words_file = 'MasterDictionary/positive-words.txt'
    negative_words_file = 'MasterDictionary/negative-words.txt'

    with open(positive_words_file, 'r', encoding='latin-1') as file:
        positive_words = set(file.read().splitlines())

    with open(negative_words_file, 'r', encoding='latin-1') as file:
        negative_words = set(file.read().splitlines())


    # Read existing Excel file provided for output saving
    excel_file = 'Output Data Structure.xlsx'
    df = pd.read_excel(excel_file)


    # Iterate through each URL_ID in the Excel file
    for index, row in df.iterrows():
        filename = row['URL_ID']+'.txt'
        text_filepath = os.path.join('', filename)

        try:
            # Attempt to open the text file
            with open(text_filepath, 'r', encoding='utf-8') as file:
                text = file.read()
        except FileNotFoundError:
            # If the text file does not exist, skip this row
            print(f"Text file '{filename}' not found. Skipping.")
            continue


        # Calculating Scores: Positivity, Negativity:
        positivity_score, negativity_score = calculate_scores(text, positive_words, negative_words)

        # Updating Scores:
        df.at[index, 'POSITIVE SCORE'] = positivity_score
        df.at[index, 'NEGATIVE SCORE'] = negativity_score

        # Calculating Polarity Score:
        polarity_score = calculate_polarity_score(positivity_score, negativity_score)

        #Updating Polarity Score:
        df.at[index, 'POLARITY SCORE'] = polarity_score

        # Calculating word count:
        total_words = len(text.split())

        # Updating word count:
        df.at[index, 'WORD COUNT'] = total_words

        # Calculate Subjectivity Score
        subjectivity_score = calculate_subjectivity_score(positivity_score, negativity_score, total_words)

        # Updating Subjectivity Scores:
        df.at[index, 'SUBJECTIVITY SCORE'] = subjectivity_score


    # Write updated DataFrame back to the Excel file
    df.to_excel(excel_file, index=False)


except Exception as e:
    error_message = str(e)
    print("An error occurred:", error_message)


else:
    print("Succeded")

Text file 'blackassign0049.txt' not found. Skipping.
Succeded


## 2. Calculating:
#### a)AVG SENTENCE LENGTH
#### b)PERCENTAGE OF COMPLEX WORDS
#### c)SYLLABLE COUNT
#### d)AVG WORDS PER SENTENCE
#### e)TOTAL COMPLEX WORD COUNT
#### f)SYLLABLE PER WORD
#### g)PERSONAL PRONOUNS
#### h)AVG WORD LENGTH

In [ ]:
try:
    # Load CMU Pronouncing Dictionary for syllable counting (source: StackOverFlow)
    pronouncing_dict = cmudict.dict()


    # Function to count syllables in a word using CMU Pronouncing Dictionary
    def count_syllables(word):
        if word.lower() in pronouncing_dict:
            return max([len(list(y for y in x if y[-1].isdigit())) for x in pronouncing_dict[word.lower()]])
        # If word not found in CMU Pronouncing Dictionary, return 1 (assume one syllable)
        return 1


    # Function to calculate Average Number of Words Per Sentence
    def calculate_avg_words_per_sentence(text):
        sentences = sent_tokenize(text)
        total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
        total_sentences = len(sentences)
        if total_sentences != 0:
            avg_words_per_sentence = total_words / total_sentences
        else:
            avg_words_per_sentence = 0
        return avg_words_per_sentence


    # Function to calculate Complex Word Count
    def calculate_complex_word_count(text):
        words = word_tokenize(text)
        complex_words = [word for word in words if len(word) > 6]  # Assume complex words are those with more than 6 characters
        return len(complex_words)


    # Function to calculate Syllables Per Word
    def calculate_syllables_per_word(text):
        words = word_tokenize(text)
        total_syllables = sum(count_syllables(word) for word in words)
        total_words = len(words)
        if total_words != 0:
            syllables_per_word = total_syllables / total_words
        else:
            syllables_per_word = 0
        return syllables_per_word


    # Function to calculate Personal Pronouns
    def calculate_personal_pronouns(text):
        personal_pronouns = ['I', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves',
                             'you', 'your', 'yours', 'yourself', 'yourselves',
                             'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself',
                             'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves']
        words = word_tokenize(text)
        personal_pronoun_count = sum(1 for word in words if word.lower() in personal_pronouns)
        return personal_pronoun_count


    # Function to calculate Average Word Length
    def calculate_avg_word_length(text):
        words = word_tokenize(text)
        total_characters = sum(len(word) for word in words)
        total_words = len(words)
        if total_words != 0:
            avg_word_length = total_characters / total_words
        else:
            avg_word_length = 0
        return avg_word_length


    # Function to calculate Average Sentence Length
    def calculate_average_sentence_length(text):
        sentences = sent_tokenize(text)
        total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
        total_sentences = len(sentences)
        if total_sentences != 0:
            average_sentence_length = total_words / total_sentences
        else:
            average_sentence_length = 0
        return average_sentence_length


    # Function to calculate Percentage of Complex Words
    def calculate_percentage_complex_words(text):
        words = word_tokenize(text)
        total_words = len(words)
        complex_words = [word for word in words if len(word) > 6]  # Assume complex words are those with more than 6 characters
        total_complex_words = len(complex_words)
        if total_words != 0:
            percentage_complex_words = total_complex_words / total_words * 100
        else:
            percentage_complex_words = 0
        return percentage_complex_words


    # Loading Output file:
    excel_file = 'Output Data Structure.xlsx'
    df = pd.read_excel(excel_file)


    # Iterating through each URL_ID
    for index, row in df.iterrows():
        text_file = row['URL_ID']+".txt"
        text_filepath = os.path.join('', text_file)
        try:
            with open(text_filepath, 'r', encoding='latin-1') as file:
                text = file.read()
        except FileNotFoundError:
            print(f"Text file '{text_file}' not found. Skipping.")
            continue

        try:

            #Start Calculating
            avg_words_per_sentence = calculate_avg_words_per_sentence(text)
            complex_word_count = calculate_complex_word_count(text)
            syllables_per_word = calculate_syllables_per_word(text)
            personal_pronouns = calculate_personal_pronouns(text)
            avg_word_length = calculate_avg_word_length(text)
            average_sentence_length = calculate_average_sentence_length(text)
            percentage_complex_words = calculate_percentage_complex_words(text)
            fog_index = 0.4 * (average_sentence_length + percentage_complex_words)


            # Updating rows
            df.at[index, 'AVG NUMBER OF WORDS PER SENTENCE'] = avg_words_per_sentence
            df.at[index, 'COMPLEX WORD COUNT'] = complex_word_count
            df.at[index, 'SYLLABLE PER WORD'] = syllables_per_word
            df.at[index, 'PERSONAL PRONOUNS'] = personal_pronouns
            df.at[index, 'AVG WORD LENGTH'] = avg_word_length
            df.at[index, 'AVG SENTENCE LENGTH'] = average_sentence_length
            df.at[index, 'PERCENTAGE OF COMPLEX WORDS'] = percentage_complex_words
            df.at[index, 'FOG INDEX'] = fog_index


        except Exception as e:
            print(f"An error occurred while processing text file '{text_file}': {str(e)}")
            continue


    df.to_excel(excel_file, index=False)

except Exception as e:
    print("An error occurred:", str(e))
else:
    print("Succeded")

Text file 'blackassign0049.txt' not found. Skipping.
Succeded
